In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
df = pd.read_csv('TWO_CENTURIES_OF_UM_RACES.csv')

C:\Users\Grzegorz\AppData\Local\Temp\ipykernel_18520\107228863.py:1: DtypeWarning: Columns (11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('TWO_CENTURIES_OF_UM_RACES.csv')


In [3]:
df.sample(5)

,Year of event,Event dates,Event name,Event distance/length,Event number of finishers,Athlete performance,Athlete club,Athlete country,Athlete year of birth,Athlete gender,Athlete age category,Athlete average speed,Athlete ID
258801,2018,27.05.2018,Trail dell'Orsa (ITA),50km,142,6:31:41 h,AIM Gruppo Sportivo,ITA,1974.0,M,M40,7.659,10532
2483841,2019,07.12.2019,Carrera x Montaña Sierra de Chiva (ESP),61km,174,8:28:16 h,NaN,ESP,1976.0,M,M40,7.201,346825
5841196,2013,22.12.2013,Shei-Pa ultramarathon (TPE),58km,422,7:57:40 h,NaN,TPE,1963.0,M,M50,7.285,13073
3350136,2022,25.06.2022,On The Rocks Trail Run (USA),50km,28,7:27:11 h,"*Mcmurray, PA",USA,1962.0,M,M60,6.709,258406
7141058,1987,07.03.1987,Six Foot Track (AUS),46.6km,111,5:46:34 h,*Bossley Park,AUS,1942.0,M,M45,8068.0,1070214


TODO: Get month from event date for seasonal analysis

Get country from event name for geolocation analysis

Calculate athelete age at the time of the event

Remove outliers from averege speed

Change athelete perfmormance dtype into something usable

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7461195 entries, 0 to 7461194
Data columns (total 13 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   Year of event              int64  
 1   Event dates                object 
 2   Event name                 object 
 3   Event distance/length      object 
 4   Event number of finishers  int64  
 5   Athlete performance        object 
 6   Athlete club               object 
 7   Athlete country            object 
 8   Athlete year of birth      float64
 9   Athlete gender             object 
 10  Athlete age category       object 
 11  Athlete average speed      object 
 12  Athlete ID                 int64  
dtypes: float64(1), int64(3), object(9)
memory usage: 740.0+ MB


## Data Preparation

In [5]:
pd.set_option('display.max_rows', None)
df['Year of event'].value_counts()

Year of event
2019    732207
2018    661483
2017    604638
2016    540073
2015    480722
2022    471598
2014    417613
2021    375604
2013    348829
2012    298981
2011    236397
2020    224203
2010    207315
2009    169969
2008    136333
2007    123766
2006    105639
2005     94253
2004     91813
2003     84114
2000     76563
2002     69190
2001     66071
1999     58672
1998     52782
1997     48538
1996     46962
1991     43954
1995     42503
1993     40793
1994     38906
1990     38775
1992     38603
1989     38491
1988     37351
1986     33014
1987     32301
1984     30692
1985     29417
1983     27542
1982     24847
1981     22577
1980     18948
1979     15790
1978     14562
1977     12781
1976      9791
1975      8498
1974      6447
1973      5938
1972      4693
1971      3984
1970      2674
1969      2213
1968      1756
1967      1359
1966      1109
1965       984
1964       830
1963       513
1962       434
1961       334
1960       270
1892       226
1959       196
1956       

In [6]:
df['Event distance/length'].value_counts()

Event distance/length
50km                   1522609
100km                   951742
50mi                    352181
56km                    333253
87km                    212062
89km                    187987
24h                     183108
100mi                   173884
60km                    159494
45km                    157799
55km                    131047
6h                      125875
12h                     114342
80km                     97962
52km                     92611
65km                     90525
70km                     82684
53km                     71828
75km                     62811
48km                     61596
46km                     60038
90km                     59493
63km                     56926
51km                     52648
72km                     52367
54km                     51406
47km                     41177
58km                     39100
67km                     37387
57km                     37034
68km                     36551
73km             

In [7]:
# Rows with year of event < 1970 will be dropped, because there's very few of them
print(f"Before dropping {df[df['Year of event']<1970].shape}")
df = df.drop(df[df['Year of event'] < 1970].index)
print(f"After dropping {df[df['Year of event']<1970].shape}")

Before dropping (12995, 13)
After dropping (0, 13)


In [8]:
df.isnull().sum()

Year of event                      0
Event dates                        0
Event name                         0
Event distance/length           1053
Event number of finishers          0
Athlete performance                2
Athlete club                 2823234
Athlete country                    3
Athlete year of birth         584744
Athlete gender                     7
Athlete age category          581527
Athlete average speed            223
Athlete ID                         0
dtype: int64

In [9]:
# Percentage of missing values
(df.isnull().mean() * 100).round(2)

Year of event                 0.00
Event dates                   0.00
Event name                    0.00
Event distance/length         0.01
Event number of finishers     0.00
Athlete performance           0.00
Athlete club                 37.90
Athlete country               0.00
Athlete year of birth         7.85
Athlete gender                0.00
Athlete age category          7.81
Athlete average speed         0.00
Athlete ID                    0.00
dtype: float64

In [10]:
# Removing club because it's useless
df.drop(['Athlete club'],axis=1,inplace=True)
# Removing age category, because it's redundant when we can get age column
df.drop(['Athlete age category'],axis=1, inplace=True)

In [11]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Use iterative imputer to predict missing values
imputer = IterativeImputer()
df['Athlete year of birth'] = imputer.fit_transform(df[['Athlete year of birth']])

In [12]:
df['Athlete year of birth'] = df['Athlete year of birth'].astype(int)

In [13]:
df['Athlete age'] = df['Year of event'] - df['Athlete year of birth']